# Codalab scoring program

## File organization

Names: `Ti_ens_exp_XXX_fov_yyy.txt`  and  `Ti_st_exp_XXX_fov_yyy.txt`, where `i` refers to the track and can be 1 (video track) or 2 (trajectory track)


In [2]:
#| export
import pandas as pd
import re
import numpy as np
from sklearn.metrics import f1_score
import sys
import os

In [3]:
#| export
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith(('.','_')):
            yield f

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from andi_datasets.utils_challenge import error_Ensemble_dataset, _get_error_bounds, load_file_to_df, error_SingleTraj_dataset

# Error bounds
threshold_error_alpha, threshold_error_D, threshold_error_s, threshold_cp = _get_error_bounds()

In [6]:
# Saving variables

# Videos
t1_ens = {'alpha': [],
          'D': []}

t1_st = {'RMSE': [],
         'JI': [],
         'alpha': [],
         'D': [],
         'state': []}

# Trajectories
t2_ens = {'alpha': [],
          'D': []}

t2_st = {'RMSE': [],
         'JI': [],
         'alpha': [],
         'D': [],
         'state': []}

In [7]:
#input_dir = sys.argv[1]
#output_dir = sys.argv[2]

input_dir = 'C:/Users/Gorka/github/ANDI_datasets/testing/codalab_data'
output_dir = 'C:/Users/Gorka/github/ANDI_datasets/testing/codalab_data'


submit_dir = os.path.join(input_dir, 'pred')
truth_dir = os.path.join(input_dir, 'true')

if not os.path.isdir(submit_dir):
    print( "%s doesn't exist", truth_dir)

In [8]:
if os.path.isdir(submit_dir) and os.path.isdir(truth_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Extracts all files in reference directory
    true_files_list = sorted(list(listdir_nohidden(truth_dir)))#os.listdir(truth_dir)

    
    for filename in true_files_list:
        track = int(filename[1])
        task = re.search('_(.+?)_labs', filename).group(1)
        exp = re.search('exp_(.+?)_', filename).group(1)
        fov = re.search('fov_(.+?).', filename).group(1)
                        
        true_file = os.path.join(truth_dir, filename)
        corresponding_submission_file = os.path.join(submit_dir, filename)
        
        if not os.path.isfile(corresponding_submission_file):
            raise FileNotFoundError(f'Prediction file for: track {track}, task {task}, experiment {exp} and FOV {fov} not found.')
        
        # Ensemble
        if task == 'ens':
            
            true = np.loadtxt(true_file, skiprows=1, delimiter = ';')
            pred = np.loadtxt(corresponding_submission_file, skiprows=1, delimiter = ';')
            
            mae_alpha, mae_D, = error_Ensemble_dataset(true_data = true,
                                                       pred_data = pred)
            
            if track == 1:
                t1_ens['alpha'].append(mae_alpha)
                t1_ens['D'].append(mae_D)
            if track == 2:                
                t2_ens['alpha'].append(mae_alpha)
                t2_ens['D'].append(mae_D)
                
        # Single trajectory
        if task == 'st':    
            df_true = load_file_to_df(true_file)
            df_pred = load_file_to_df(corresponding_submission_file)

            rmse_CP, JI, error_alpha, error_D, error_s = error_SingleTraj_dataset(df_true = df_true, df_pred = df_pred, 
                                                                                  threshold_error_alpha = threshold_error_alpha,
                                                                                  threshold_error_D = threshold_error_D, 
                                                                                  threshold_error_s = threshold_error_s,
                                                                                  threshold_cp = threshold_cp,
                                                                                  prints = False, disable_tqdm = True)
            
            if track == 1:
                t1_st['RMSE'].append(rmse_CP)
                t1_st['JI'].append(JI)
                t1_st['alpha'].append(error_alpha)
                t1_st['D'].append(error_D)
                t1_st['state'].append(error_s)                
            if track == 2:
                t2_st['RMSE'].append(rmse_CP)
                t2_st['JI'].append(JI)
                t2_st['alpha'].append(error_alpha)
                t2_st['D'].append(error_D)
                t2_st['state'].append(error_s)
                
        print(f'Track {track}, Task {task}, Exp {exp}, FOV {fov}: OK!')

Track 1, Task ens, Exp 0, FOV 0: OK!
Track 1, Task ens, Exp 1, FOV 0: OK!
Track 1, Task st, Exp 0, FOV 0: OK!
Track 1, Task st, Exp 1, FOV 0: OK!
Track 2, Task ens, Exp 0, FOV 0: OK!
Track 2, Task ens, Exp 1, FOV 0: OK!
Track 2, Task st, Exp 0, FOV 0: OK!
Track 2, Task st, Exp 1, FOV 0: OK!


In [9]:
df_true.shape

(65, 5)

In [46]:
# Define output file
output_filename = os.path.join(output_dir, 'scores.txt')
output_file = open(output_filename, 'w')

### Saving single trajectory data

# we define a variable that gives the worst values for each metric
worst_value_st = {'RMSE': threshold_cp,
                  'JI': 0,
                  'alpha': threshold_error_alpha,
                  'D': threshold_error_D,
                  'state': 0}


for key in t1_st: 
    
    if key in ['RMSE', 'alpha', 'D']:
        save_t1 = np.nanmin([np.mean(t1_st[key]),
                          worst_value_st[key]])
        save_t2 = np.nanmin([np.mean(t2_st[key]),
                          worst_value_st[key]])
    elif key in ['JI', 'state']:
        save_t1 = np.nanmax([np.mean(t1_st[key]),
                          worst_value_st[key]])
        save_t2 = np.nanmax([np.mean(t2_st[key]),
                          worst_value_st[key]])
        
    
    output_file.write('T1_st_'+ key +f' : {save_t1}\n')    
    output_file.write('T2_st_'+ key +f' : {save_t2}\n')
    
### Saving ensemble data

'''WHAT ARE THE THRESHOLDS FOR THIS?'''
worst_value_ens = {'alpha': 100,
                  'D': 100}

for key in t1_ens:
    save_t1 = np.nanmin([np.mean(t1_ens[key]),
                          worst_value_ens[key]])
    save_t2 = np.nanmin([np.mean(t2_ens[key]),
                         worst_value_ens[key]])    
    
    
    output_file.write('T1_ens_'+ key +f' : {save_t1}\n')    
    output_file.write('T2_ens_'+ key +f' : {save_t2}\n')
    
output_file.close()